In [27]:
import pandas as pd

In [28]:
#import data 
samsung = pd.read_csv('samsung.csv')
samsung.head(3)

,Unnamed: 0,timestamp,text,model
0,0,2020-05-13 23:59:55+00:00,"Samsung CJ89 43"" Curved UltraWide 3840 X 1200 ...",samsung
1,1,2020-05-13 23:59:40+00:00,Kkkkkkkkkkkkkkk fala mal do meu Samsung agr,samsung
2,2,2020-05-13 23:59:36+00:00,Apoyar para una buena causa Samsung Plegable y...,samsung


In [29]:
#convert columns' type to string
samsung[['text']] = samsung[['text']].astype(str)


In [30]:
#check datatype
print(samsung.dtypes)

Unnamed: 0     int64
timestamp     object
text          object
model         object
dtype: object


In [31]:
#convert character to ASCII
import unicodedata

samsung = pd.DataFrame.from_dict(samsung, orient='columns')
print(samsung['text'].apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode()))


0       Samsung CJ89 43" Curved UltraWide 3840 X 1200 ...
1             Kkkkkkkkkkkkkkk fala mal do meu Samsung agr
2       Apoyar para una buena causa Samsung Plegable y...
3       magina vc cagar de investir em propaganda num ...
4           a samsung respondendo o luiza kkkkkkkkk tudo 
5        Samsung Galaxy A21s  4   Exynos 850  #Samsung...
6        Samsung Galaxy A21s  4   Exynos 850  #Samsung...
7       samsung eh o fundo mais bonito mas o design do...
8       Je vais te retirer ton Samsung tout suite imef...
9        I can't wait to play on my samsung smart fridge!
10      coz what we facing today pandenic that's reaso...
11      Eu acho que era so samsung que era assim Quand...
12      They say some kpop guy like he isnt one the am...
13      O papai tem um dos melhores Samsungs do mercad...
14      Samsung Galaxy A51 outsold all other Android p...
15      sim, eu tambem comprei um A50 faz pouco tempo ...
16                         samsung 11 pro yun HAHAHAHAHA 
17      ...   

In [32]:
import re

In [33]:
#samsung.drop('Unnamed: 0', axis=1, inplace=True)

samsung = samsung[~samsung['text'].isnull()]

def preprocess(text):
    text =  text.str.replace("(<br/>)", "")
    text =  text.str.replace('(<a).*(>).*(</a>)', '')
    text =  text.str.replace('(&amp)', '')
    text =  text.str.replace('(&gt)', '')
    text =  text.str.replace('(&lt)', '')
    text =  text.str.replace('(\xa0)', ' ')  
    return  text
samsung['text'] = preprocess(samsung['text'])

samsung['review_len'] = samsung['text'].astype(str).apply(len)
samsung['word_count'] = samsung['text'].apply(lambda x: len(str(x).split()))


In [34]:
samsung.head(10)

,Unnamed: 0,timestamp,text,model,review_len,word_count
0,0,2020-05-13 23:59:55+00:00,"Samsung CJ89 43"" Curved UltraWide 3840 X 1200 ...",samsung,198,19
1,1,2020-05-13 23:59:40+00:00,Kkkkkkkkkkkkkkk fala mal do meu Samsung agr,samsung,43,7
2,2,2020-05-13 23:59:36+00:00,Apoyar para una buena causa Samsung Plegable y...,samsung,162,18
3,3,2020-05-13 23:59:30+00:00,magina vc cagar de investir em propaganda num ...,samsung,260,42
4,4,2020-05-13 23:59:19+00:00,a samsung respondendo o luiza kkkkkkkkk tudo,samsung,45,7
5,5,2020-05-13 23:59:15+00:00,หลุดภาพเรนเดอร์ Samsung Galaxy A21s พร้อมกล้อง...,samsung,210,15
6,6,2020-05-13 23:59:14+00:00,หลุดภาพเรนเดอร์ Samsung Galaxy A21s พร้อมกล้อง...,samsung,210,15
7,7,2020-05-13 23:59:11+00:00,samsung eh o fundo mais bonito mas o design do...,samsung,68,14
8,8,2020-05-13 23:58:59+00:00,Je vais te retirer ton Samsung tout suite imef...,samsung,52,10
9,9,2020-05-13 23:58:39+00:00,I can't wait to play on my samsung smart fridge!,samsung,48,10


In [35]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [36]:
samsung['review_len'].iplot(kind='hist',
    bins=100,
    xTitle='review length',
    linecolor='black',
    yTitle='count',
    title='Review Text Length Distribution')

In [37]:

def  clean_text(samsung, text_field, new_text_field_name):
    samsung[new_text_field_name] = samsung[text_field].str.lower()
    samsung[new_text_field_name] = samsung[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    samsung[new_text_field_name] = samsung[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return samsung


In [44]:
samsung_clean = clean_text(samsung, 'text','clean_text')
samsung.head(3)

,Unnamed: 0,timestamp,text,model,review_len,word_count,clean_text
0,0,2020-05-13 23:59:55+00:00,"Samsung CJ89 43"" Curved UltraWide 3840 X 1200 ...",samsung,198,19,samsung cj curved ultrawide x resolution hz...
1,1,2020-05-13 23:59:40+00:00,Kkkkkkkkkkkkkkk fala mal do meu Samsung agr,samsung,43,7,kkkkkkkkkkkkkkk fala mal do meu samsung agr
2,2,2020-05-13 23:59:36+00:00,Apoyar para una buena causa Samsung Plegable y...,samsung,162,18,apoyar para una buena causa samsung plegable y...


In [45]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

samsung_clean['clean_text'] = samsung['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
samsung.head(3)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sidratulmuntaha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,timestamp,text,model,review_len,word_count,clean_text
0,0,2020-05-13 23:59:55+00:00,"Samsung CJ89 43"" Curved UltraWide 3840 X 1200 ...",samsung,198,19,samsung cj curved ultrawide x resolution hz mo...
1,1,2020-05-13 23:59:40+00:00,Kkkkkkkkkkkkkkk fala mal do meu Samsung agr,samsung,43,7,kkkkkkkkkkkkkkk fala mal meu samsung agr
2,2,2020-05-13 23:59:36+00:00,Apoyar para una buena causa Samsung Plegable y...,samsung,162,18,apoyar para una buena causa samsung plegable p...


In [46]:
samsung_clean.isnull().sum()

Unnamed: 0    0
timestamp     0
text          0
model         0
review_len    0
word_count    0
clean_text    0
dtype: int64

In [47]:
samsung_clean.dropna()

,Unnamed: 0,timestamp,text,model,review_len,word_count,clean_text
0,0,2020-05-13 23:59:55+00:00,"Samsung CJ89 43"" Curved UltraWide 3840 X 1200 ...",samsung,198,19,samsung cj curved ultrawide x resolution hz mo...
1,1,2020-05-13 23:59:40+00:00,Kkkkkkkkkkkkkkk fala mal do meu Samsung agr,samsung,43,7,kkkkkkkkkkkkkkk fala mal meu samsung agr
2,2,2020-05-13 23:59:36+00:00,Apoyar para una buena causa Samsung Plegable y...,samsung,162,18,apoyar para una buena causa samsung plegable p...
3,3,2020-05-13 23:59:30+00:00,magina vc cagar de investir em propaganda num ...,samsung,260,42,magina vc cagar de investir em propaganda num ...
4,4,2020-05-13 23:59:19+00:00,a samsung respondendo o luiza kkkkkkkkk tudo,samsung,45,7,samsung respondendo luiza kkkkkkkkk tudo
5,5,2020-05-13 23:59:15+00:00,หลุดภาพเรนเดอร์ Samsung Galaxy A21s พร้อมกล้อง...,samsung,210,15,samsung galaxy exynos samsung galaxyas mobileocta
6,6,2020-05-13 23:59:14+00:00,หลุดภาพเรนเดอร์ Samsung Galaxy A21s พร้อมกล้อง...,samsung,210,15,samsung galaxy exynos samsung galaxyas mobileocta
7,7,2020-05-13 23:59:11+00:00,samsung eh o fundo mais bonito mas o design do...,samsung,68,14,samsung eh fundo mais bonito mas design iphone...
8,8,2020-05-13 23:58:59+00:00,Je vais te retirer ton Samsung tout suite imef...,samsung,52,10,je vais te retirer ton samsung tout suite imefier
9,9,2020-05-13 23:58:39+00:00,I can't wait to play on my samsung smart fridge!,samsung,48,10,cant wait play samsung smart fridge


In [48]:
samsung_clean.to_csv('samsung_clean.csv')